In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Define Simulation Parameters
# Initial investment amount (X)
INITIAL_CAPITAL = 10000.0

# Define the stocks and their capital weights (must sum to 1.0)
TICKERS = ['META', 'MSFT', 'NVDA', 'AAPL', 'AMZN', 'GOOG']
WEIGHTS = np.array([0.0855, 0.1649, 0.0624, 0.0010, 0.0602, 0.626]) # Must sum to 1.0

# Define the comparison index
COMPARISON_INDEX = 'NQ=F'

# Simulation period: Lump sum investment 10 years ago
end_date_string = '2025-01-01'

END_DATE = datetime.strptime(end_date_string, '%Y-%m-%d')
START_DATE = END_DATE - timedelta(days=10 * 365)

# Risk-Free Rate (Annualized)
ANNUAL_RISK_FREE_RATE = 0.02  # 2.0%

# Trading days per year for annualization
TRADING_DAYS_PER_YEAR = 252

# Data Retrieval
def get_stock_data(tickers, index_ticker, start_date, end_date):
    """Downloads adjusted close prices for tickers and the comparison index."""
    all_tickers = tickers + [index_ticker]
    print(f"Downloading historical data for {all_tickers}...")

    format_string = "%Y-%m-%d"
    data = yf.download(all_tickers, start=start_date.strftime(format_string), end=end_date.strftime(format_string), auto_adjust=True)['Close']

    # Filter the data to the defined START_DATE and END_DATE from parameters
    data_filtered = data.loc[start_date:end_date].dropna(how='all')

    stock_data = data_filtered[tickers]
    index_data = data_filtered[index_ticker]

    # Drop any remaining rows with NaN values in the stock data
    stock_data.dropna(inplace=True)
    index_data.dropna(inplace=True)

    return stock_data, index_data

stock_prices, index_prices = get_stock_data(TICKERS, COMPARISON_INDEX, START_DATE, END_DATE)

# Portfolio and Index Balance Calculation & Return Alignment
# Calculate daily returns
daily_returns = stock_prices.pct_change().dropna()
index_returns = index_prices.pct_change().dropna()

# Align both return series to the common set of trading days for fair comparison
common_index = daily_returns.index.intersection(index_returns.index)
daily_returns = daily_returns.loc[common_index]
index_returns = index_returns.loc[common_index]

# Calculate the daily portfolio return: (Weighted sum of individual stock returns)
portfolio_returns = (daily_returns * WEIGHTS).sum(axis=1)

# Calculate Portfolio Balance Growth
initial_value = pd.Series(WEIGHTS * INITIAL_CAPITAL, index=TICKERS)
# Rescale cumulative returns to the aligned index
cumulative_returns = (1 + daily_returns).cumprod()
portfolio_balance_growth = (cumulative_returns * initial_value).sum(axis=1)

# Calculate Index Balance Growth
index_cumulative_growth = (1 + index_returns).cumprod()
# Scale the index cumulative growth by the initial capital
index_balance_growth = index_cumulative_growth * INITIAL_CAPITAL / index_cumulative_growth.iloc[0]


# Performance Ratio Calculations
# Convert annual risk-free rate to daily rate
daily_risk_free_rate = (1 + ANNUAL_RISK_FREE_RATE)**(1/TRADING_DAYS_PER_YEAR) - 1

def max_drawdown(returns):
    """Calculates the Maximum Drawdown (MDD)."""
    # Use returns to calculate cumulative wealth index
    cumulative = (1 + returns).cumprod()
    peak = cumulative.expanding(min_periods=1).max()
    drawdown = (cumulative / peak) - 1
    return drawdown.min()

def calculate_sharpe_ratio(returns, rf_rate, annual_factor):
    """Calculates the annualized Sharpe Ratio."""
    excess_returns = returns - rf_rate
    # The ratio is (mean excess return / standard deviation of excess return) * sqrt(annual factor)
    if excess_returns.std() == 0:
         return np.nan
    return excess_returns.mean() / excess_returns.std() * np.sqrt(annual_factor)

def calculate_sortino_ratio(returns, rf_rate, annual_factor):
    """Calculates the annualized Sortino Ratio."""
    target = rf_rate
    # Only consider negative (downside) returns relative to the risk-free rate
    downside_returns = returns[returns < target]
    downside_deviation = downside_returns.std()

    if downside_deviation == 0:
        return np.nan

    return (returns.mean() - rf_rate) / downside_deviation * np.sqrt(annual_factor)

def calculate_calmar_ratio(returns, annual_factor):
    """Calculates the Calmar Ratio."""
    # Annualized Compound Annual Growth Rate (CAGR)
    total_return = (1 + returns).prod()
    num_years = len(returns) / annual_factor
    # Handle the case where the investment period is less than one year
    if num_years < 0.1: # Arbitrary small threshold
        return np.nan

    cagr = total_return**(1/num_years) - 1

    mdd = max_drawdown(returns)

    # Calmar Ratio = CAGR / |Max Drawdown|
    if abs(mdd) == 0:
        return np.nan

    return cagr / abs(mdd)

# Calculate the metrics for the Portfolio
p_sharpe = calculate_sharpe_ratio(portfolio_returns, daily_risk_free_rate, TRADING_DAYS_PER_YEAR)
p_sortino = calculate_sortino_ratio(portfolio_returns, daily_risk_free_rate, TRADING_DAYS_PER_YEAR)
p_calmar = calculate_calmar_ratio(portfolio_returns, TRADING_DAYS_PER_YEAR)

# Calculate the metrics for the Index (New)
i_sharpe = calculate_sharpe_ratio(index_returns, daily_risk_free_rate, TRADING_DAYS_PER_YEAR)
i_sortino = calculate_sortino_ratio(index_returns, daily_risk_free_rate, TRADING_DAYS_PER_YEAR)
i_calmar = calculate_calmar_ratio(index_returns, TRADING_DAYS_PER_YEAR)

# Plotting and Results
plt.figure(figsize=(12, 6))
# Plot Portfolio
portfolio_balance_growth.plot(label='Portfolio Growth', legend=True)
# Plot Index
index_balance_growth.plot(label=f'{COMPARISON_INDEX} Growth', legend=True, linestyle='--')

plt.title(f'Portfolio vs. Index ({COMPARISON_INDEX}) Growth (Initial: ${INITIAL_CAPITAL:,.2f})')
plt.xlabel('Date')
plt.ylabel('Balance ($)')
plt.grid(True)
plt.show()

# Print results
print("\n" + "="*70)
print(f"Simulation Period: {portfolio_returns.index[0].strftime('%Y-%m-%d')} to {portfolio_returns.index[-1].strftime('%Y-%m-%d')}")
print(f"Initial Investment: ${INITIAL_CAPITAL:,.2f}")
print("="*70)

# Quantitative Comparison Table
comparison_data = {
    'Metric': ['Final Balance', 'Total Return', 'Sharpe Ratio', 'Sortino Ratio', 'Calmar Ratio'],
    'Portfolio': [
        f"${portfolio_balance_growth.iloc[-1]:,.2f}",
        f"{((portfolio_balance_growth.iloc[-1] / INITIAL_CAPITAL) - 1) * 100:.2f}%",
        f"{p_sharpe:.4f}",
        f"{p_sortino:.4f}",
        f"{p_calmar:.4f}"
    ],
    f'{COMPARISON_INDEX}': [
        f"${index_balance_growth.iloc[-1]:,.2f}",
        f"{((index_balance_growth.iloc[-1] / INITIAL_CAPITAL) - 1) * 100:.2f}%",
        f"{i_sharpe:.4f}",
        f"{i_sortino:.4f}",
        f"{i_calmar:.4f}"
    ]
}

df_comparison = pd.DataFrame(comparison_data)

print(df_comparison.to_markdown(index=False))
print("="*70)